## 使用卷积神经网络实现文本情感分类

In [1]:
import tensorflow.keras as keras
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import os
import sys
import time
import tensorflow as tf

for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

matplotlib 3.4.3
numpy 1.18.5
pandas 1.3.2
sklearn 0.24.2
tensorflow 2.3.0
tensorflow.keras 2.4.0


### 1. 数据读取与预处理

#### 1.1 读取数据

In [2]:
# imdb 是电影评论数据集, 类别为积极与消极
imdb = keras.datasets.imdb
# 词表数量, 前 10000 个会被保存, 其他的会被当做特殊字符处理
vocab_size = 10000
# 词表的index从index_from 开始算, 因为要设置特殊字符! 0 ~ 3 是特殊字符
index_from = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size, index_from=index_from)

# 打印向量
print(train_data[0], train_labels[0])
print(train_data.shape, train_labels.shape)
print(len(train_data[0]), len(train_data[1]))

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32] 1
(25000,) (25000,)
218 189


In [3]:
print(test_data.shape, test_labels.shape)

(25000,) (25000,)


#### 1.2 构建词表

In [4]:
# 获取词表， word_index = (单词, 索引)
word_index = imdb.get_word_index()
print(len(word_index))

88584


In [5]:
print(type(word_index))
# 打印前10个词表
for word, index in word_index.items():
    if int(index) < 30:
        print(word , index)

<class 'dict'>
with 16
one 28
have 25
i 10
he 26
as 14
it 9
is 6
in 8
but 18
on 20
of 4
his 24
all 29
this 11
not 21
you 22
a 3
for 15
be 27
br 7
the 1
was 13
and 2
to 5
film 19
movie 17
that 12
are 23


In [6]:
# 因为之前的设置 index_from = 3, 所以所有的索引都需要偏移3
word_index = {word : (index + 3) for word,index in word_index.items()}

In [7]:
# 设置特殊字符
# <PAD> padding 的填充字符
# <START> 句子的开头的填充字符
# <UNK> 找不到字符时, 返回UNK
# <END> 句子的末尾的特殊字符

word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<END>'] = 3
reverse_word_index = dict([(index, word) for word, index in word_index.items()])

In [8]:
# 使用词表反向解析句子
def decode_review(text_ids):
    return " ".join(
        [reverse_word_index.get(word_id, "<UNK>") for word_id in text_ids]
    )
# 解析数字为真实文本
decode_review(train_data[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh